### Importing necessary libraries

In [13]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

#Date time
import datetime as dt
from datetime import timedelta, date

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd
import geopandas

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

# Planetary Computer Tools
import pystac
import pystac_client
import stackstac
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
import contextily
pc.settings.set_subscription_key('b0067a12405d4fd4a4cc82d28869d9bc')

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()

#Scaling using Dask
import dask_gateway
import dask
import xarray as xr
import dask.array as da

In [5]:
crop_presence_data = pd.read_csv("./Data/Crop_Location_Data_20221201.csv")
crop_presence_data.head()

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


### Creating bounding boxes from the given coordinate

In [6]:
def calculate_bbox(lat_long, box_size_deg=0.0004):
    lat_long=lat_long.replace('(','').replace(')','').replace(' ','').split(',')
    
    min_lon = float(lat_long[1]) - box_size_deg/2
    min_lat = float(lat_long[0])- box_size_deg/2
    max_lon = float(lat_long[1]) + box_size_deg/2
    max_lat = float(lat_long[0]) + box_size_deg/2
    
    return min_lon, min_lat, max_lon, max_lat

In [7]:
time_of_interest = '2021-12-01/2022-12-01'
resolution = 10  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for crs=4326 

In [8]:
rvi_df = pd.DataFrame()
bbox = []
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    bbox.append(calculate_bbox(coordinates))

bbox_data = pd.DataFrame(bbox,columns =['min_lon', 'min_lat', 'max_lon', 'max_lat'])

100%|████████████████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 262581.64it/s]


#### Sample code to explore STAC items

In [9]:
@dask.delayed
def create_rvi_for_each_aoi(time_of_interest, bbox):
    items = []
    rvi = []
    #Creating catalog
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_of_interest)
    items = search.get_all_items()
    print(len(items))
    
    # Load the data using Open Data Cube
    data = stac_load(items,bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox, chunks={"x": 5120, "y": 5120}, crs="EPSG:4326", resolution=scale)
    #print(type(data))
    

    # Calculate the mean of the data across the sample region
    #mean_aoi = data.mean(dim=['latitude','longitude'])
    #print('Mean polarization', mean_aoi)

    #Calculating mean vv and vh of our area of interest
    #vh = mean_aoi["vh"]
    #vv = mean_aoi["vv"]
    #print('Vertical-vertical polarisation', vv)

    #print('Vertical-horizontal polarisation', vh)
    # Calculate RVI
    #dop = (mean_aoi.vv / (mean_aoi.vv + mean_aoi.vh))
    #m = 1 - dop
    #rvi = (np.sqrt(dop))*((4*mean_aoi.vh)/(mean_aoi.vv + mean_aoi.vh))

    return data

##### Accessing images as array and dataset

In [39]:
import xarray as xr

ds = xr.Dataset()

In [40]:
for i in range(1):
    data= create_rvi_for_each_aoi(time_of_interest, bbox_data.iloc[i])

#### Lazily append all datasets

In [41]:
lazy_datasets = []
for i,item in bbox_data.iterrows():
    ds = create_rvi_for_each_aoi(time_of_interest, item)
    lazy_datasets.append(ds)

datasets = dask.compute(*lazy_datasets)

6161
61
61
61

61
61
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
61
61
61
61
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
61
61
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
61
61
61
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
61
61
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
6

61
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
61
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
61
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
61
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
61
61
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xa

61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
61
61
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
<class 'xarray.core.dataset.Dataset'>
61
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarray.core.dataset.Dataset'>
61
<class 'xarra

In [11]:
lazy_datasets = []
for i in range(4):
    ds = create_rvi_for_each_aoi(time_of_interest, bbox_data.iloc[i])
    lazy_datasets.append(ds)

datasets = dask.compute(*lazy_datasets)

61
61
61
61


In [26]:
DS = xr.concat(datasets, dim='time')

In [21]:
DS

<xarray.Dataset>
Dimensions:      (longitude: 21, time: 61, latitude: 21)
Coordinates:
  * longitude    (longitude) float64 105.3 105.3 105.3 ... 105.3 105.3 105.3
  * time         (time) datetime64[ns] 2021-12-04T22:46:07.919581 ... 2022-11...
  * latitude     (latitude) float64 10.32 10.32 10.32 ... 10.32 10.32 10.32
    spatial_ref  int32 4326
Data variables:
    vv           (time, latitude, longitude) float32 dask.array<chunksize=(1, 5, 21), meta=np.ndarray>
    vh           (time, latitude, longitude) float32 dask.array<chunksize=(1, 5, 21), meta=np.ndarray>

In [27]:
# Calculate the mean of the data across the sample region
mean_aoi = DS.mean(dim=['latitude','longitude'])
#print('Mean polarization', mean_aoi)

#Calculating mean vv and vh of our area of interest
vh = mean_aoi["vh"]
vv = mean_aoi["vv"]
#print('Vertical-vertical polarisation', vv)

#print('Vertical-horizontal polarisation', vh)
# Calculate RVI
dop = (mean_aoi.vv / (mean_aoi.vv + mean_aoi.vh))
m = 1 - dop
rvi = (np.sqrt(dop))*((4*mean_aoi.vh)/(mean_aoi.vv + mean_aoi.vh))

In [30]:
vv

<xarray.DataArray 'vv' (time: 244)>
dask.array<mean_agg-aggregate, shape=(244,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 2021-12-04T22:46:07.919581 ... 2022-11...
    spatial_ref  int32 4326

In [29]:
rvi = rvi.persist()

In [31]:
rvi

<xarray.DataArray (time: 244)>
dask.array<mul, shape=(244,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 2021-12-04T22:46:07.919581 ... 2022-11...
    spatial_ref  int32 4326